In [2]:
import torch
import torch.nn as nn
from torch.nn import LayerNorm
#空间dropout层,一种常见于CNN中的操作,可以帮助防止模型出现过拟合
#提高模型性能和泛化能力
class spatial_drop_out(nn.Dropout2d):
    def __init__(self, p, inplace: bool = False) -> None:
        super().__init__(p, inplace)
    def forward(self, x) -> torch.Tensor:
        x.unsqueeze_(2)         #输入张量[batch_size,seq_len,embedding_size] -> [batch_size,seq_len,,1,embedding_size]
        x.permute_(0,3,2,1)     #重新排列[batch_size,embedding_size,seq_len,1]
        x = super(spatial_drop_out,self).forward(x) #先dropout一次
        x.permute_(0,3,2,1)     #再转换回来
        x.squeeze_(2)           #最后降回初始维度
        return x
#单任务类NER (只输出BIOSlabel的情况)
#NER数据集一般为标注好的文本和文本中实体出现的起始和结束位置
class Bilstm_Crf(nn.Module):
    def __init__(self,vocab_size,embedding_size,hidden_size,BISO_labels,drop_p):
        super().__init__()
        self.embedding_size = embedding_size        #由处理好的数据得来
        #nn.embedding()
        self.embedding = nn.Embedding(vocab_size,embedding_size)    #建立词向量层,随模型训练得到新的词向量,将每个单词和其对应词向量作为字典储存下来
        self.bilstm = nn.LSTM(                                      #传入初始化的词向量层作为输入
            input_size = embedding_size,
            hidden_size = hidden_size,
            batch_first = True,
            num_layers = 2,     #bi-lstm,每层又各有两个子层
            dropout = drop_p,   #lstm内部的drop_out
            bidirectional = True
        )
        self.dropout = spatial_drop_out(drop_p) #lstm之后再一个全局drop 
        #归一化层用来加速收敛/提高泛化能力/避免梯度爆炸/消失
        self.layer_norm = LayerNorm(hidden_size *2) #接一个层归一化层, x2是因为bi-lstm的输出维度是普通lstm的两倍,nlp中一般就用layernorm
        self.classifier = nn.Linear(hidden_size*2, len(BISO_labels))    #线性分类层,这里输出的就是emssion矩阵
        self.crf =  CRF(num_tags = len(bios_labels),batch_first = True) #pytorch 没有内置的CRF模块,但是一般常见的CRF模型输入都是 标签长度,batch_first

    def forward(self,input_ids,input_mask,bio_label = None):
        embs = self.embeddings(input_ids)       #从input token生成初始层向量
        embs = self.dropout(embs)               
        #这里这个input_mask是一个1/0张量,是通过遍历text2id实现的,有单词的就填1,没有的就填0(或者有实际意义的就填1(比如start/end tag),没有意义的就填0(长度pad))
        #最后的input_mask应该和text2id的形状一模一样,只是里面元素为1/0   .float() -> 把这个1/0的bool张量转为浮点数,unsqueeze(2)增加维度
        #即增加一个embedding_size这个维度,和初始化的embedding对齐
        embs = embs*input_mask.float().unsqueeze(2)     #对没有意义的padding token id进行掩码,避免没有意义的计算,从而提高模型效率,增强性能
        #lstm的输出为Ct,Ht, t代表t节点,Ct就表示当前节点/词/字时的输出,Ht是隐藏态输出,用来流入下一个节点继续计算
        #一般都会提取Ct而忽略Ht,下面的写法就是因为同时有两个输出,但我们只需要其中一个的时候,再命名变量的时候就用 _ 来省略掉
        sequence_output,_ = self.bilstm(embs)   
        sequence_output = self.layer_norm(sequence_output)      
        features = self.classifier(sequence_output)
        #下面参数后加了一个逗号,用来告诉python这是一个只有一个元素的元组(tuple),而不是一个单一元素
        #意思就是 outputs是(featurs,-1*loss),这样可以追踪每词训练得到的损失和特征向量
        outputs = (features,)
        if bio_label is not None:
            #计算crf损失
            loss = self.crf(emissimons = features,tag = bio_label,mask = input_mask)
            #-1 乘以损失,为了反向传播最大化梯度
            outputs = (-1*loss,) + outputs      #元组的拼接操作, (-1*loss,) + (feature) = (-1*loss,feature)
        return outputs

#网络搭建完毕之后,就是数据加载,优化器设置
#之后设置训练
for epoch in tqdm(range(args.epochs), desc='Epoch'):
    for step, batch in enumerate(train_dataloader):
        model.train()
        batch = tuple(t.to(args.device) for t in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'bio_labels': batch[3]}

        outputs = model(**inputs)
        biesos_logits = outputs[1]

        loss = outputs[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

        scheduler.step()
        optimizer.step()
        model.zero_grad()
        global_step += 1
#解码得到模型预测序列
inputs = {'input_ids': batch[0], 'input_mask': batch[1], "bio_labels": batch[2]}
outputs = model(**inputs)
loss, biesos_logits = outputs[0], outputs[1]
biesos_tags = model.crf.decode(biesos_logits,inputs['input_mask'])
biesos_tags = biesos_tags.squeeze(0).cpu().numpy().tolist()



#多任务形NER
#输出两个序列:BIO标签 和 对应实体类型(组织/公司等)
#对应的就有两个loss,两个序列训练时共享权重
#下面采用Bert+CRF的模型来实现




In [29]:
#用BERT搭建:
import torch
import tqdm
from transformers import BertTokenizerFast, BertForTokenClassification
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import os   
# Load pre-trained model tokenizer (vocabulary)
#加载预训练模型的tokenizer（词汇表）,其中包含了词汇到索引的映射关系
#通过char_level=True可以将句子中的每个字视为一个token,因为是中文所以设置为true
#fast和普通的tokenizer的区别在于fast的tokenizer可以并行处理，而普通的tokenizer只能单线程处理
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese',char_level=True)
#max_len 指定了输入模型的最大长度，超过这个长度的句子将被截断
max_len = 128
#假设我们的数据采用BIO标,有三类实体 loc,org,per
#定义标签到索引的映射关系
tag2idx = {'O':0,'B-per':1,'I-per':2,'B-org':3,'I-org':4,'B-loc':5,'I-LOC':6}
nonword_label_id = -100     #标签序列的padding值,选一个不常用的就行
#定义模型
#这里我们使用BertForTokenClassification模型,它可以同时预测每个token的标签   
#通过tokenizer.convert_tokens_to_ids可以将文本转化为索引序列
#BERT内常见的特殊token有 [CLS] [SEP] [PAD] [UNK]
#他们分别代表 句子的开始 [CLS] 和句子的结束 [SEP] 以及未登录词 [UNK], [PAD] 是长度padding
len_pad_id,cls_id,sep_id,unk_id = tokenizer.convert_tokens_to_ids(['[PAD]','[CLS]','[SEP]','[UNK]'])
#假设我们的数据集已经是字典形式,k,v分别词/字和对应的标签
#e.g: {'今':'O','天':'O','我':'B-per'}
#通过下面的函数处理:
def data2input(text2dict,max_len):
    temp = []
    for k,v in data:
        k = [tokenizer.convert_tokens_to_ids(i) for i in k] #把句子中的字转化为索引
        v = [tag2idx[v]] #把标签转化为索引
        #创建输入序列和标签序列
        input_ids = [cls_id] + k
        label_seq = [nonword_label_id] + v
        #如果句子长度超过max_len,则截断
        if len(input_ids) > max_len - 1:
            input_ids = input_ids[:max_len - 1]
            label_seq = label_seq[:max_len - 1]
        #添加[SEP]和padding
        input_ids.append(sep_id)
        label_seq.append(nonword_label_id)
        cur_len = len(input_ids)
        #如果句子长度小于max_len,则添加padding
        #而对于标签序列,则把小于长度的部分填充为nonword_label_id
        if cur_len < max_len:
            input_ids += [len_pad_id] * (max_len - cur_len)
            label_seq += [nonword_label_id] * (max_len - cur_len)

        #token_type_ids 用于区分句子的起止，0表示前半部分，1表示后半部分
        #BERT 最初是为序列对任务(如问答、自然语言推理等)而设计的,这些任务需要将两个句子序列连接成一个输入序列。token_type_ids 的作用就是区分这两个句子序列,使模型能够分别编码它们。
        #序列标注任务中,我们只有一个输入句子序列,不需要区分两个句子。所以token_type_ids可以全部设置为0。
        token_type_ids = [0] * max_len
        #attention_mask 用于区分padding的部分，1表示非padding部分，0表示padding部分
        #相似的,由于是NER任务,我们只用告诉模型有词的地方,其余地方都设为0
        attention_mask = [1] * cur_len + [0] * (max_len - cur_len)
        #将数据转化为tensor
        temp.append((np.array(input_ids),np.array(token_type_ids),np.array(attention_mask),np.array(label_seq)))
        return temp

#处理好的数据转化为batch/训练和测试集
processed_data = data2input(data,max_len)
train_data = processed_data[:int(len(processed_data)*0.8)]      #训练集
#训练集用于模型的训练,测试集用于最终评估模型的性能。\
#但是如果只依赖测试集来调整模型超参数和进行模型选择,很容易导致过拟合测试集。
#因此需要从训练集中分出一部分作为验证集,用于模型选择和调参,避免过度依赖测试集。
#有时训练集和测试集的数据分布会有差异,这种情况下使用验证集可以更好地估计模型在测试集上的真实表现
dev_data = train_data[int(len(train_data)*0.6):]        #抽训练集20%做验证集
test_data = processed_data[int(len(processed_data)*0.8):]        #测试集

#batch 大小设置
batch_size = 16
num_of_workers = 1
#载入进dataloader
train_dataloader = DataLoader(train_data,batch_size=batch_size,num_workers=num_of_workers,shuffle=True)
dev_dataloader = DataLoader(dev_data,batch_size=batch_size,num_workers=num_of_workers,shuffle=False)
test_dataloader = DataLoader(test_data,batch_size=batch_size,num_workers=num_of_workers,shuffle=False)



#模型搭建
#从huggingface加载crf
from transformers import BertForTokenClassification
from torchcrf import CRF
class Bert_CRF(nn.Module):
    def __init__(self,num_of_labels):
        super(Bert_CRF, self).__init__()
        #它基于预训练的BERT模型,在输出端添加了一个线性层,将BERT的输出映射到标签空间,从而可以对每个token进行分类标注。
        #这个线性分类层的输出可以直接作为CRF层的输入,从而实现序列标注任务。不需要softmax层。
        self.bert = BertForTokenClassification.from_pretrained('bert-base-chinese',num_labels=num_of_labels)
        self.crf = CRF(num_of_labels,batch_first=True)
    def forward(self,input_ids,token_type_ids,attention_mask,label_seq):
        #bert输出的特征向量
        emmissions = self.bert(input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask)[0]
        #转存为crf的输入
        crf_inputs = (emmissions,)
        if label_seq is not None:
            #计算crf损失
            loss = self.crf(emmissions = crf_inputs,tag = tag2idx,mask = attention_mask)
            #-1 乘以损失,为了反向传播最大化梯度
            crf_inputs = (-1*loss,) + crf_inputs      #元组的拼接操作, (-1*loss,) + (feature) = (-1*loss,feature)
        return crf_inputs   #(loss,feature)

      

#冻结bert中的其他层,只训练最后一层线性层
#这一行代码遍历了模型的所有可训练参数,并将名称(n)包含"layer.11"字符串的参数名称存储在decay_layers列表中。在BERT模型中,"layer.11"对应于最后一层的参数。
decay_layers = [n for n,p in model.named_parameters() if layer.11 in n]
crf_params = list(ner_model.crf.parameters())     #获取crf层的参数
optimizer_grouped_parameters = [
    #这个字典包含了所有不在decay_layers列表中的参数,也就是除了最后一层之外的所有参数。对于这些参数,权重衰减值设置为0.01。
    {'params': [p for n,p in model.named_parameters() if not any(nd in n for nd in decay_layers)],'weight_decay': 0.01},
    #这个字典包含了decay_layers列表中的参数,也就是最后一层的参数。对于这些参数,权重衰减值设置为0.0,也就是不进行权重衰减。
    {'params': [p for n,p in model.named_parameters() if any(nd in n for nd in decay_layers)],'weight_decay': 0.0}
    #这个字典包含了crf层的参数,权重衰减值设置为0.01。
    {'params': crf_params, 'weight_decay': 0.01}
]


#实例化模型,转用cuda运行
model = Bert_CRF(len(tag2idx))
model.to('cuda')
#定义优化器
optimizer = torch.optim.AdamW(optimizer_grouped_parameters,lr=2e-5)
#开始训练过程,首先训练n个epoch,然后在验证集上评估模型的性能,选择最优的模型保存。
#先从epoch开始循环
for epoch in range(10):
    #训练模型,初始化准确率和损失
    total_correct,total_loss = 0,0
    total_pred = 0
    #进入训练模式
    model.train()
    #遍历batch数据
    for input_ids,token_type_ids,attention_mask,label_seq in tqdm(train_dataloader):
        input_ids,token_type_ids,attention_mask,label_seq = input_ids.to('cuda'),token_type_ids.to('cuda'),attention_mask.to('cuda'),label_seq.to('cuda')
        #优化器初始化梯度
        optimizer.zero_grad()
        #计算crf的loss,logits和模型输出一样,想不起来了就看模型forwad函数返回的是啥
        loss,logits = model(input_ids,token_type_ids,attention_mask,label_seq)
        total_loss += loss.item()       #累计每个batch的损失
        #logits.shape[0]是batch大小,logit是模型预测的标签序列,label_seq是真实标签序列
        #遍历batch内数据,计算准确率和损失
        for i in range(logits.shape[0]):
            logits_clean = logits[i][label_seq[i]!=-100]     #去除padding部分的预测结果
            preds = logits_clean.argmax(dim=1)               #从每个token生成的emmision矩阵中选取最大的索引(标签)作为预测结果,(预测序列)
            label_seq_clean = label_seq[i][label_seq[i]!=0]  #去除padding部分的真实标签(真实序列)
            correct = (preds == label_seq_clean).sum().item() #正确率
            total_correct += correct
            total_pred += label_seq_clean.size

        #反向传播
        loss.backward()
        optimizer.step()
        #本轮准确率
        train_acc = total_correct/total_pred
        #本轮平均损失
        train_loss = total_loss/len(train_dataloader)
        print(f'Epoch {epoch+1}, Training Loss: {train_loss}, Training Acc: {train_acc}')

  #验证模型,步骤和训练一样,只是换了数据集
    model.eval()
    total_loss_validation = 0
    total_correct_validation = 0
    total_predicted_validation = 0
    with torch.no_grad():
        for input_ids, token_type_ids, attention_mask, label_seq in tqdm(dev_dataloader):
            input_ids, token_type_ids, attention_mask, label_seq = input_ids.to('cuda'), token_type_ids.to('cuda'), attention_mask.to('cuda'), label_seq.to('cuda')
            loss, logits = model(input_ids, token_type_ids, attention_mask, label_seq)
            total_loss_validation += loss.item()
            # 计算每个batc的准确率
            for i in range(logits.shape[0]):
                logits_clean = logits[i][label_seq[i] != -100]  # 过滤掉padding
                preds = logits_clean.argmax(dim=1)
                label_seq_clean = label_seq[i][label_seq[i] != -100]
                correct = (preds == label_seq_clean).sum().item()
                total_correct_validation += correct
                total_predicted_validation += label_seq_clean.size

    val_acc = total_correct_validation / total_predicted_validation
    val_loss = total_loss_validation / len(dev_dataloader)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss}, Validation Acc: {val_acc}')

#模型保存
#需要先把模型转回到cpu,然后保存
model.cpu()
saved_path = ...  # 保存路径
torch.save(model.state_dict(),'saved_path' + 'bert_crf.pt')
#模型加载
model = Bert_CRF(len(tag2idx))
model.load_state_dict(torch.load('saved_path' + 'bert_crf.pt'))
if torch.cuda.is_available():
    model.to('cuda')
#在测试集上评估模型的性能
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for input_ids, token_type_ids, attention_mask, label_seq in tqdm(test_dataloader):
        input_ids, token_type_ids, attention_mask, label_seq = input_ids.to('cuda'), token_type_ids.to('cuda'), attention_mask.to('cuda'), label_seq.to('cuda')
        loss, logits = model(input_ids, token_type_ids, attention_mask, label_seq)
        all_preds.extend(logits.argmax(dim=2).cpu().numpy().tolist())
        all_labels.extend(label_seq.cpu().numpy().tolist())
# 计算性能指标
precision, recall, f1 = calculate_metrics(all_preds, all_labels, tag2idx)

# 打印性能指标
for tag, value in f1.items():
    print(f'Tag: {tag}, Precision: {precision[tag][0] / precision[tag][1]}, Recall: {recall[tag][0] / recall[tag][1]}, F1 Score: {f1[tag]:.4f}')

# 计算整体的准确率、召回率和F1分数
overall_precision = sum(precision.values(), [])[0] / sum(precision.values(), [])[1]
#整体召回率是一个衡量模型在所有类别上识别正例的能力的指标。它是模型性能的一个重要指标，特别是在类别不平衡的情况下，可以帮助我们了解模型是否倾向于漏检某些类别的正例。
overall_recall = sum(recall.values(), [])[0] / sum(recall.values(), [])[1]
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall + 1e-8)

print(f'Overall Precision: {overall_precision:.4f}')
print(f'Overall Recall: {overall_recall:.4f}')
print(f'Overall F1 Score: {overall_f1:.4f}')


#除了简单的准确率,根据业务需求,我们还需要计算每个标签的准确率,f1-score等指标,这可以通过计算每个标签的准确率来实现。
#下面的函数计算每对(预测序列/真实序列)中每种标签的预测准确率。
def calculate_tag_accuracies(preds, label_seq_clean, tag2idx):
    """
    计算每个标签的准确率。

    参数:
    preds (torch.Tensor): 模型预测的标签，形状为 (batch_size, seq_length)。
    label_seq_clean (torch.Tensor): 真实的标签，形状为 (batch_size, seq_length)，已经过滤掉padding。
    tag2idx (dict): 标签到索引的映射字典。

    返回:
    dict: 一个字典，键为标签名，值为对应的准确率。
    """
    # 计算所有标签的正确预测
    correct = (preds == label_seq_clean).float()

    # 计算每个标签的预测总数
    tag_totals = label_seq_clean.ne(-100).float().sum(dim=1)

    # 初始化一个字典来存储每个标签的准确率
    tag_accuracies = {}

    # 对于每个标签，计算准确率
    for tag in tag2idx:
        tag_index = tag2idx[tag]
        tag_preds = preds == tag_index
        tag_correct = correct * tag_preds

        # 计算当前标签的正确预测数和总预测数
        tag_correct_count = tag_correct.sum().item()
        tag_total_count = tag_totals.sum().item()

        # 计算准确率并存储到字典中
        if tag_total_count > 0:  # 防止除以0
            tag_accuracy = tag_correct_count / tag_total_count
            tag_accuracies[tag] = tag_accuracy

    return tag_accuracies

# 假设你已经有了模型的预测结果preds和过滤padding后的真实标签label_seq_clean
# 你可以这样调用函数：
# tag_accuracies = calculate_tag_accuracies(preds, label_seq_clean, tag2idx)

# 计算准确率、召回率和F1分数
def calculate_metrics(all_preds, all_labels, tag2idx):
    unique_tags = list(tag2idx.keys())
    precision, recall, f1 = {tag: [0, 0] for tag in unique_tags}  # 初始化为[正确数量, 总数]

    for tag in unique_tags:
        # 计算每个标签的TP, FP, FN
        TP = sum((all_preds == tag) & (all_labels == tag))
        FP = sum((all_preds == tag) & (all_labels != tag))  # 预测为tag的样本中，实际上不是tag的数量
        FN = sum((all_preds != tag) & (all_labels == tag))  # 实际上为tag的样本中，预测不是tag的数量

        # 更新precision, recall
        #精确率是指在所有被模型预测为特定类别的样本中，实际属于该类别的样本所占的比例。
        #就是被预测为某个tag的token中,有多少真的是这个tag的
        precision[tag] = [TP, sum(all_labels == tag)]
        #召回率是指在所有实际为特定类别的样本中，被模型预测为该类别的样本所占的比例。
        #就是被本来是某个tag的token中,有多少被预测为这个tag的
        recall[tag] = [TP, sum(all_preds == tag)]  # [正确数量, 总数]

    # 计算F1分数
    for tag in unique_tags:
        if precision[tag][1] != 0 and recall[tag][1] != 0:
            f1[tag] = 2 * (precision[tag][0] * recall[tag][0]) / (precision[tag][0] + recall[tag][0])
        else:
            f1[tag] = 0

    return precision, recall, f1

In [ ]:
import torch.utils.data.dataset as Dataset
import torch.utils.data.dataloader as DataLoader
import numpy as np
import pandas as pd
#这里演示一下数据集的载入
#pytorch中的数据集一般通过两个类载入,一个是Dataset类,定义数据集的各种方法
#一种是DataLoader,定义载入的实现
#对于处理好的数据,即已经是所需维度的张量:
# 把数据放在数据库中
x = torch.tensor(data[..])#切片选出特征列/行
y = torch.tensor(data[...])     #选出标签行
torch_dataset = Data.TensorDataset(x, y)  # 对给定的 tensor 数据，将他们包装成 dataset
loader = Data.DataLoader(
    # 从数据库中每次抽出batch size个样本
    dataset=torch_dataset,       # torch TensorDataset format
    batch_size=BATCH_SIZE,       # mini batch size
    shuffle=True,                # 要不要打乱数据 (打乱比较好)
    num_workers=2,               # 多线程来读数据
    collate_fn = collate_fn      # 数据转换,一般需要自己根据数据重新写方法,如果不需要特殊转换就不用这个参数
)


#一个collate_fn的例子
#常用方法包括通过重写collate_fn方法来动态定义batch内序列长度
#即每个batch都以当前batch内最长的序列来做padding,这样做可以有效减少内存
'''
def collate_fn(batch):
    # 假设batch是一个列表,其中每个元素都是一个元组(sequence, label)
    # 例如: batch = [([1, 2, 3], 0), ([4, 5], 1), ([6, 7, 8, 9], 0)]
    
    # 分离序列和标签
    sequences, labels = zip(*batch)
    
    # 对序列进行padding
    padded_sequences = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    
    # 创建掩码张量
    lengths = [len(seq) for seq in sequences]
    masks = torch.bernoulli(torch.ones(padded_sequences.shape)).bool()
    for i, length in enumerate(lengths):
        masks[i, length:] = False
    
    return padded_sequences, torch.tensor(labels), masks

'''



#最常见的方法还是需要自己定义这两个类的子类:
#先继承父类 Dataset,再重写 init/len/getitem这几个方法(一般只用改前两个)
class my_dataset(Dataset):
    def __init__(self,dataset_path):        #输入文件地址加载数据集,预处理 ->筛选行列->分开特征和标签列等预处理都可以在这里面做
        self.data = pd.read(dataset_path)
        '''
        raw_data = pd.read()
        self_data = ...
        self_labeldata = ....
        '''
    def __len__(self):
        return len(self.data)         #返回数据集长度
    def __getitem__(self,index):
        sub_data = self.data.values[index]  #按索引返回数据集
        '''
        self_data = self.data.values[index] 
        self_labeldata = self.labeldata.values[index] 
        '''

        return sub_data,labeldata

test_data = my_dataset(file_path)
